<a href="https://colab.research.google.com/github/gowrijp/Projects-Random/blob/master/Generating%20Fake%20Trump%20Tweets/trump_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
import os

In [2]:
from google.colab import files
uploaded = files.upload()

Saving trump_tweets_all.txt to trump_tweets_all.txt


In [3]:
# load and lightly pre-process data
text = " ".join(open("trump_tweets_all.txt").readlines())
text = " ".join(text.split())
text = text.encode("ascii", errors="ignore").decode()
print(text[:100])

OBAMAGATE! Proud of Mike! https://t.co/g1HvItV79N First time in 22 years. Mike will be a GREAT CONGR


In [4]:
# calculate the vocabulary (number of unique characters in text)
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

91 unique characters


In [0]:
# mapping from unique characters to indices
char_to_index = {u: i for i, u in enumerate(vocab)}
index_to_char = np.array(vocab)

In [0]:
# numerical representation of text
text_as_int = np.array([char_to_index[c] for c in text])

# define the sequence length, which will determine how many example pairs per epoch
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)


In [0]:
def split_input_target(chunk):
    """Split a chunk of length n+1 into a tuple containing the the input (first n chars)
        and the target (last n chars) """
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text


In [0]:
# create the dataset
BATCH_SIZE = 64
BUFFER_SIZE = 10000
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
dataset = sequences.map(split_input_target)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

vocab_size = len(vocab)
embedding_dim = 64
rnn_units = 512

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    """Define the model: character embedding -> GRU -> fully connected """
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [13]:
train = True
if train:
    model = build_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)
    model.summary()

    def loss(labels, logits):
        return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

    # attach optimizer and loss
    model.compile(optimizer='adam', loss=loss)

    # Set up directory for saving checkpoints of the model
    checkpoint_dir = './training_checkpoints'
    checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

    checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_prefix,
        save_weights_only=True)

    EPOCHS = 30
    history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 64)            5824      
_________________________________________________________________
gru_1 (GRU)                  (64, None, 512)           887808    
_________________________________________________________________
dense_1 (Dense)              (64, None, 91)            46683     
Total params: 940,315
Trainable params: 940,315
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
664/664 [==============================] - 716s 1s/step - loss: 2.4719
Epoch 2/30
664/664 [==============================] - 713s 1s/step - loss: 1.7053
Epoch 3/30
664/664 [==============================] - 735s 1s/step - loss: 1.5143
Epoch 4/30
664/664 [==============================] - 746s 1s/step - loss: 1.4339
Epoch 5/30
664/664 [==============================

In [0]:
def generate_text(model, start_string):
    """Generate text, given a trained model and a starting string"""
    num_generate = 280
    input_eval = [char_to_index[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(index_to_char[predicted_id])

    return start_string + ''.join(text_generated)

In [15]:
generate = True
if generate:
    # load the model back from a checkpoint
    checkpoint_dir = './training_checkpoints'
    model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
    model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
    model.build(tf.TensorShape([1, None]))
    print(generate_text(model, start_string="Hillary"))

Hillary 'fought it) you frantil very player. @Speakeficial is raising soon. #CelebApprentice .@TravinSchwatt3i Mid Republicans and more it is made all proud of you! @MiamiHerald "Yen early adding I wish is such a complete lists to reele--in Iowa would do a special time to the @pbitated 
